In [1]:
from pulp import *
# initialize the model
prob = LpProblem("linenmgmt", LpMinimize)

demand = {
    0 : 229,
    1 : 236,
    2 : 223,
    3 : 270,
    4 : 261,
    5 : 232,
    6 : 220,
    7 : 250,
    8 : 236,
    9 : 235,
    10: 284,
    11: 286,
    12: 262,
    13: 275,
    14: 271,
    15: 246,
    16: 244,
    17: 288,
    18: 283,
    19: 248,
    20: 259,
    21: 265,
    22: 256,
    23: 254,
    24: 286,
    25: 286,
    26: 270,
    27: 251,
    28: 247,
    29: 217,
    30: 211,
    31: 272,
    32: 279
    }

T = 31
bedsheetPurchaseCost = 181.5
pillowPurchaseCost = 64.35
bedsheetWashCost = 5
pillowWashCost = 3.3
othercost = 400 + 1700

# ---------------------
# VARIABLES
# ---------------------

bdshtpurind = LpVariable.dicts("bdshtpurind", range(1, T), 0, None, cat = LpBinary)
pillowpurind = LpVariable.dicts("pillowpurind", range(1, T), 0, cat = LpBinary) 
bdshtpur = LpVariable.dicts("bdshtpur", range(1, T), 0, None)
pillowpur = LpVariable.dicts("pillowpur", range(1, T), 0) 

bdshtout = LpVariable.dicts("bdshtout", range(1, T), 0, None)
pillowout = LpVariable.dicts("pillowout", range(1, T), 0) 


pillowrecycl = LpVariable.dicts("pillowrecycl", range(1, T), 0, None)

unusedbdsht = LpVariable.dicts("unusedbdsht", range(1, T), 0, None)
unusedpillow = LpVariable.dicts("unusedpillow", range(1, T), 0, None) 


# Constraints

# Period 1
prob += bdshtout[1] >= 2 * 1.02 * (demand[0]) 
prob += bdshtpur[1] + 1260 >= 2 * 1.02 * (demand[0] + demand[1] + demand[2])
prob += pillowout[1] >= 1.02 * (demand[0]) 
prob += pillowpur[1] + 680 >= 1.02 * (demand[0] + demand[1] + demand[2])

prob += unusedbdsht[1] >= bdshtpur[1] + 1260 - bdshtout[1]
prob += unusedpillow[1] >= pillowpur[1] + 680 - pillowout[1]
prob += pillowrecycl[1] <= 0.001 * 1.02 * 2 * demand[0] 
prob += bdshtout[1] == 2 * pillowout[1]

# Period 2
prob += bdshtout[2] >= 2 * 1.02 * (demand[1]) 
prob += bdshtpur[2] + unusedbdsht[1] + 470 >= 2 * 1.02 * (demand[1] + demand[2] + demand[3])
prob += pillowout[2] >= 1.02 * (demand[1]) 
prob += pillowpur[2] + unusedpillow[1] + 240 >= 1.02 * (demand[1] + demand[2] + demand[3])

prob += unusedbdsht[2] >= bdshtpur[2] + unusedbdsht[1] + 470 - bdshtout[2]
prob += unusedpillow[2] >= pillowpur[2] + unusedpillow[1] + 240 - pillowout[2]
prob += pillowrecycl[2] <= 0.001 * 1.02 * 2 * demand[1] 
prob += bdshtout[2] == 2 * pillowout[2]

# Period 3 onwards
for i in range(3, T):
    prob += bdshtout[i] >= 2 * 1.02 * (demand[i - 1]) 
    prob += bdshtpur[i] + unusedbdsht[i - 1] + (0.9972 * 2 * 1.02 * demand[i-3]) >= 2 * 1.02 * (demand[i - 1] + demand[i] + demand[i + 1])
    prob += pillowout[i] >= 1.02 * (demand[i - 1]) 
    prob += pillowpur[i] + unusedpillow[i - 1] + (0.9972 * 1.02 * demand[i-3]) >= 1.02 * (demand[i - 1] + demand[i] + demand[i + 1])

    prob += unusedbdsht[i] >= bdshtpur[i] + unusedbdsht[i-1] - bdshtout[i] + (0.9972 * 2 * 1.02 * demand[i-3])
    prob += unusedpillow[i] >= pillowpur[i] + unusedpillow[i - 1]  - pillowout[i] + pillowrecycl[i - 2] + (0.9972 * 1.02 * demand[i-3])
    prob += pillowrecycl[i] <= 0.001 * 1.02 * 2 * demand[i - 1] 
    prob += bdshtout[i] == 2 * pillowout[i]
    
for i in range(1, T):
    prob += bdshtpur[i] <= 100000 * bdshtpurind[i]
    prob += pillowpur[i] <= 100000 * pillowpurind[i]

prob += sum([bdshtpurind[i] for i in range(1, T)]) <= 1
prob += sum([pillowpurind[i] for i in range(1, T)]) <= 1


# Objective function
prob += (sum([bdshtpur[i] * bedsheetPurchaseCost for i in range(1,T)]) + 
        sum([pillowpur[i] * pillowPurchaseCost for i in range(1,T)]) + 
        sum([bdshtout[i] * bedsheetWashCost for i in range(1,T)]) + 
        sum([pillowout[i] * pillowWashCost for i in range(1,T)]) + 
        sum([pillowrecycl[i] * pillowWashCost for i in range(1,T)]) +
        sum([400 * demand[i] for i in range(0,T-1)])), "Objective"
    
    
prob.writeLP("linenmgmt.lp")
    
status = prob.solve(GLPK(options=["--ranges","linenmgmt.sen"]))
print(status)

#print the result
print("bdshtpur")
for i in range(1, T):
    print(bdshtpur[i].value())


print("pillowpur")       
for i in range(1, T):
    print(pillowpur[i].value())
print("bdshtout")
for i in range(1, T):
    print(bdshtout[i].value())


print("pillowout")       
for i in range(1, T):
    print(pillowout[i].value())

print("pillowrecycl")   
for i in range(1, T):
    print(pillowrecycl[i].value())

    
print("unusedbdsht")   
for i in range(1, T):
    print(unusedbdsht[i].value())
    
print("unusedpillow")    
for i in range(1, T):
    print(unusedpillow[i].value())

print("bdshtpurind")   
for i in range(1, T):
    print(bdshtpurind[i].value())
    
print("pillowpurind")    
for i in range(1, T):
    print(pillowpurind[i].value())
    
# The optimised objective function value is printed to the screen
print ("Total Cost of Linen Maintenance = ", prob.objective.value())

1
bdshtpur
143.52
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
pillowpur
21.76
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
bdshtout
467.16
481.44
454.92
550.8
532.44
473.28
448.8
510.0
481.44
479.4
579.36
583.44
534.48
561.0
552.84
501.84
497.76
587.52
577.32
505.92
528.36
540.6
522.24
518.16
583.44
583.44
550.8
512.04
503.88
442.68
pillowout
233.58
240.72
227.46
275.4
266.22
236.64
224.4
255.0
240.72
239.7
289.68
291.72
267.24
280.5
276.42
250.92
248.88
293.76
288.66
252.96
264.18
270.3
261.12
259.08
291.72
291.72
275.4
256.02
251.94
221.34
pillowrecycl
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
unusedbdsht
1017.16
1072.31
1083.24
1012.53
933.738
1009.72
1091.87
1092.66
1133.63
1217.19
1200.86
1095.48
1138.74
1159.54
1139.69
1197.28
1250.81
1163.72
1082.77
1162.72
1210

In [40]:
#%load aavinMilk.sen


pillowrecycl_1